# **Deep Hedging**
# Buchkov Viacheslav

In [1]:
from pathlib import Path

from deep_hedging import ExperimentConfig, EuropeanCall, seed_everything
from deep_hedging.dl import SpotDataset, Trainer, Assessor
from deep_hedging.dl.models import LSTMHedger
from deep_hedging.dl.baselines import BaselineEuropeanCall

RANDOM_SEED = 12

In [2]:
config = ExperimentConfig(
    RANDOM_SEED=RANDOM_SEED,
    DATA_ROOT=Path("data"),
    OUTPUT_ROOT=Path(".")
)
seed_everything(RANDOM_SEED)

config.DEVICE

In [3]:
config

## Dataset.

In [4]:
spot_dataset = SpotDataset(
    n_days=config.N_DAYS,
    instrument_cls=EuropeanCall
)
spot_dataset.df.head()

In [5]:
trainer = Trainer(
    model_cls=LSTMHedger,
    instrument_cls=EuropeanCall,
    config=config
)
trainer.run(1)

In [6]:
assessor = Assessor(
    model=trainer.hedger,
    baseline=BaselineEuropeanCall(dt=trainer.dt).to(config.DEVICE),
    test_loader=trainer.test_loader
)
assessor.run()

In [7]:
trainer.save(config.OUTPUT_ROOT)